In [30]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()


In [31]:
xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
def init_nn_model(dimensions):
    nn_model = Sequential()
    nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=dimensions, activation='relu' ))
    nn_model.add(Dropout(rate=0.2))
    nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
    nn_model.add(Dropout(rate=0.1))
    nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)

# Dataset 1

In [32]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'1-x.csv')
y_path = os.path.join(proccessed_data_path,'1-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(20)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002804
                     0.000898
                     0.000058
                     0.023010
                     0.033061
                     0.004596
                     0.000396
                     0.028683
                     0.010656
                     0.000343
                     0.010176
                     0.009461
                     0.000268
                     0.001290
                     0.004175
                     0.000132
                     0.007296
                     0.000202
                     0.000226
                     0.000029
Iteration 2


...

 NN - feature_importance_vals
                     0.001609
                     0.011815
                     0.002368
                     0.013095
                     0.058278
                     0.009017
                     0.013691
                     0.009571
                     0.005572
                     0.004593
                     0.005863
                     0.009413
                     0.006057
                     0.006119
                     0.005834
                     0.000610
                     0.036882
                     0.003915
                     0.004745
                     0.000628
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                 4.898440e-04
                 6.201420e-06
                 1.129611e-06
                 4.311527e-05
                 3.336512e-04
                 4.445478e-05
                 9.727420e-06
                 2.125430e-03
                 1.302765e-03
                 4.155344e-06
                 3.143661e-03
                 4.675057e-04
                 1.201174e-05
                 1.995078e-05
                 6.299460e-05
                 7.641366e-07
                 9.524438e-05
                 2.162151e-06
                 2.267250e-06
                 1.241311e-06


# Dataset 2

In [35]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'2-x.csv')
y_path = os.path.join(proccessed_data_path,'2-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(19)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.003716
                     0.009166
                     0.002175
                     0.017691
                     0.023028
                     0.003009
                     0.003288
                     0.009712
                     0.005488
                     0.001763
                     0.005096
                     0.006076
                     0.002629
                     0.004516
                     0.001768
                     0.000075
                     0.025930
                     0.000705
                     0.001597
                     0.000190
Iteration 2


...

 NN - feature_importance_vals
                     0.002183
                     0.009416
                     0.002362
                     0.020378
                     0.027485
                     0.008898
                     0.003823
                     0.002103
                     0.006561
                     0.002068
                     0.007046
                     0.012440
                     0.003490
                     0.003955
                     0.005223
                     0.000220
                     0.039650
                     0.002267
                     0.001803
                     0.000666
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.001083
                     0.015725
                     0.002107
                     0.018258
                     0.032046
                     0.006879
                     0.004641
                     0.003111
                     0.008834
                     0.001262
                     0.002982
                     0.011209
                     0.002508
                     0.002668
                     0.005066
                     0.000167
                     0.025073
                     0.001654
                     0.001802
                     0.000551


# Dataset 3

In [36]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'3-x.csv')
y_path = os.path.join(proccessed_data_path,'3-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(19)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002557
                     0.010868
                     0.002753
                     0.024690
                     0.026517
                     0.010606
                     0.003783
                     0.009278
                     0.009071
                     0.002538
                     0.005975
                     0.009989
                     0.002153
                     0.004111
                     0.004498
                     0.000153
                     0.039726
                     0.001721
                     0.002010
                     0.000585
Iteration 2


...

 NN - feature_importance_vals
                     0.004084
                     0.000174
                     0.000019
                     0.004350
                     0.020793
                     0.004843
                     0.003579
                     0.002999
                     0.004104
                     0.000309
                     0.001182
                     0.007586
                     0.000035
                     0.000462
                     0.001386
                     0.000116
                     0.000287
                     0.000067
                     0.000231
                     0.000031
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.007427
                     0.010955
                     0.001490
                     0.025170
                     0.082519
                     0.011465
                     0.013125
                     0.010768
                     0.016858
                     0.001752
                     0.004023
                     0.027379
                     0.001574
                     0.002183
                     0.013042
                     0.000252
                     0.026366
                     0.001980
                     0.002859
                     0.000595


# Dataset 4

In [37]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'4-x.csv')
y_path = os.path.join(proccessed_data_path,'4-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(10)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.003559
                     0.008110
                     0.001500
                     0.021234
                     0.046163
                     0.010923
                     0.007354
                     0.008757
                     0.010378
                     0.001712
                     0.002723
                     0.014853
                     0.001310
                     0.002482
                     0.006133
                     0.000177
                     0.028169
                     0.001345
                     0.001926
                     0.000452
Iteration 2


...

 NN - feature_importance_vals
                     0.003329
                     0.004441
                     0.001094
                     0.013608
                     0.032372
                     0.009056
                     0.004949
                     0.001834
                     0.006860
                     0.001259
                     0.004985
                     0.014661
                     0.001847
                     0.001985
                     0.007705
                     0.000140
                     0.026557
                     0.001511
                     0.002429
                     0.000242
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.005324
                     0.007964
                     0.000918
                     0.011791
                     0.030550
                     0.006809
                     0.004445
                     0.001400
                     0.007051
                     0.001087
                     0.004497
                     0.012416
                     0.001197
                     0.001850
                     0.007975
                     0.000114
                     0.022642
                     0.001259
                     0.002116
                     0.000289


# Dataset 5

In [38]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'5-x.csv')
y_path = os.path.join(proccessed_data_path,'5-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(9)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002239
                     0.007965
                     0.001735
                     0.028296
                     0.041321
                     0.011743
                     0.006496
                     0.014520
                     0.011363
                     0.002694
                     0.000759
                     0.014789
                     0.001642
                     0.002696
                     0.007125
                     0.000174
                     0.033164
                     0.001362
                     0.002491
                     0.000523
Iteration 2


...

 NN - feature_importance_vals
                     0.002471
                     0.000736
                     0.000223
                     0.005673
                     0.018835
                     0.005466
                     0.003100
                     0.004723
                     0.004712
                     0.000566
                     0.000197
                     0.008640
                     0.000383
                     0.000661
                     0.002132
                     0.000116
                     0.003382
                     0.000248
                     0.000543
                     0.000090
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.007429
                     0.009021
                     0.001137
                     0.026279
                     0.074753
                     0.011499
                     0.011661
                     0.010733
                     0.016925
                     0.001918
                     0.001959
                     0.022994
                     0.001402
                     0.001791
                     0.013125
                     0.000263
                     0.022427
                     0.001482
                     0.002739
                     0.000643


# Dataset 6

In [39]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'6-x.csv')
y_path = os.path.join(proccessed_data_path,'6-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(17)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.001866
                     0.004614
                     0.000929
                     0.021109
                     0.035139
                     0.010047
                     0.005634
                     0.020141
                     0.009466
                     0.002682
                     0.000183
                     0.011281
                     0.001041
                     0.002200
                     0.005217
                     0.000235
                     0.017451
                     0.000834
                     0.001647
                     0.000402
Iteration 2


...

 NN - feature_importance_vals
                     0.002985
                     0.002627
                     0.000818
                     0.014275
                     0.026565
                     0.009629
                     0.003999
                     0.007068
                     0.008066
                     0.001340
                     0.001424
                     0.013112
                     0.001403
                     0.001897
                     0.006202
                     0.000136
                     0.019428
                     0.001053
                     0.001556
                     0.000251
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.004597
                     0.005568
                     0.000630
                     0.014385
                     0.040539
                     0.009938
                     0.006053
                     0.008462
                     0.011935
                     0.001462
                     0.002844
                     0.017956
                     0.000929
                     0.001539
                     0.008773
                     0.000129
                     0.019816
                     0.000944
                     0.001806
                     0.000236


# Dataset 7

In [40]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'7-x.csv')
y_path = os.path.join(proccessed_data_path,'7-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(13)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002739
                     0.005183
                     0.001062
                     0.025543
                     0.037755
                     0.010347
                     0.005914
                     0.016895
                     0.009362
                     0.002448
                     0.001552
                     0.012368
                     0.001028
                     0.002439
                     0.006242
                     0.000253
                     0.022429
                     0.000842
                     0.001674
                     0.000216
Iteration 2


...

 NN - feature_importance_vals
                     0.003257
                     0.003856
                     0.000666
                     0.013085
                     0.027323
                     0.009760
                     0.004077
                     0.003061
                     0.007285
                     0.000998
                     0.000904
                     0.014355
                     0.001249
                     0.002344
                     0.006821
                     0.000174
                     0.023121
                     0.000958
                     0.001389
                     0.000398
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.007336
                     0.004471
                     0.000620
                     0.020975
                     0.058933
                     0.009440
                     0.009274
                     0.011390
                     0.012317
                     0.001386
                     0.001731
                     0.021056
                     0.001014
                     0.002318
                     0.008582
                     0.000330
                     0.011989
                     0.000557
                     0.001115
                     0.000478


# Dataset 8

In [41]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'8-x.csv')
y_path = os.path.join(proccessed_data_path,'8-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(20)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002376
                     0.004153
                     0.000839
                     0.023631
                     0.037166
                     0.009943
                     0.005775
                     0.016850
                     0.008014
                     0.002267
                     0.001576
                     0.013625
                     0.001057
                     0.002783
                     0.006657
                     0.000333
                     0.019091
                     0.000544
                     0.001295
                     0.000585
Iteration 2


...

 NN - feature_importance_vals
                     0.002210
                     0.002694
                     0.000626
                     0.012457
                     0.033233
                     0.010228
                     0.004990
                     0.006692
                     0.007033
                     0.001231
                     0.000982
                     0.018322
                     0.001259
                     0.002155
                     0.008012
                     0.000177
                     0.020825
                     0.001077
                     0.001544
                     0.000390
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.006172
                     0.003046
                     0.000497
                     0.016151
                     0.047222
                     0.008010
                     0.007365
                     0.008940
                     0.009720
                     0.001027
                     0.000987
                     0.017186
                     0.000708
                     0.002317
                     0.006683
                     0.000308
                     0.008234
                     0.000532
                     0.000813
                     0.000314


# Dataset 9

In [42]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'9-x.csv')
y_path = os.path.join(proccessed_data_path,'9-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(14)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.002664
                     0.001406
                     0.000456
                     0.021955
                     0.034366
                     0.008243
                     0.005452
                     0.016053
                     0.007548
                     0.001646
                     0.001006
                     0.012152
                     0.000345
                     0.001844
                     0.004994
                     0.000249
                     0.007417
                     0.000353
                     0.000678
                     0.000221
Iteration 2


...

 NN - feature_importance_vals
                     0.001197
                     0.000024
                     0.000004
                     0.001556
                     0.009950
                     0.002241
                     0.001549
                     0.001890
                     0.002537
                     0.000137
                     0.000024
                     0.004388
                     0.000001
                     0.000488
                     0.000566
                     0.000067
                     0.000403
                     0.000016
                     0.000054
                     0.000003
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.007550
                     0.003851
                     0.000455
                     0.017844
                     0.062879
                     0.006958
                     0.009574
                     0.009217
                     0.012012
                     0.000891
                     0.001860
                     0.017609
                     0.000441
                     0.002625
                     0.008749
                     0.000321
                     0.010327
                     0.000802
                     0.000887
                     0.000244


# Dataset 10

In [43]:
#Read data
proccessed_data_path =os.path.join(os.path.pardir,'data','processed')
x_path = os.path.join(proccessed_data_path,'10-x.csv')
y_path = os.path.join(proccessed_data_path,'10-y.csv')

dfx = pd.read_csv(x_path)
dfy = pd.read_csv(y_path)

x = dfx.drop(columns=['Unnamed: 0'],axis = 'columns')
y = dfy.drop(columns=['Unnamed: 0'],axis = 'columns').values.flatten()

ii = 1
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	init_nn_model(5)
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1  

Iteration 1


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.077119
                      0.346461
                      0.215342
                      0.034753
                      0.825371
                      0.092523
                      0.000000
                      0.381300
                      0.090741
                      0.000000
                      0.229811
                      0.083327
                      0.000000
                      0.241186
                      0.184656
                      0.000000
                      0.470708
                      0.005419
                      0.009187
                      0.006410
 DT - feature_importance_vals
                     0.001618
                     0.046043
                     0.019839
                     0.000000
                     0.003607
                     0.002334
                     0.085650
                     0.016348
                     0.000831
                     0.019409
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.004156
                     0.003409
                     0.000663
                     0.022581
                     0.038914
                     0.007007
                     0.005915
                     0.013746
                     0.008095
                     0.001635
                     0.001294
                     0.011712
                     0.000558
                     0.002455
                     0.005292
                     0.000253
                     0.016001
                     0.000832
                     0.001105
                     0.000111
Iteration 2


...

 NN - feature_importance_vals
                     0.002568
                     0.002214
                     0.000588
                     0.009869
                     0.020002
                     0.006033
                     0.002881
                     0.001705
                     0.005966
                     0.000594
                     0.000056
                     0.009005
                     0.000832
                     0.002011
                     0.006058
                     0.000161
                     0.018055
                     0.001239
                     0.001178
                     0.000108
Iteration 10


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.080717
                      0.338892
                      0.197523
                      0.046504
                      0.788866
                      0.089790
                      0.000000
                      0.389750
                      0.114849
                      0.000000
                      0.225800
                      0.089945
                      0.000000
                      0.218665
                      0.176896
                      0.000000
                      0.449176
                      0.002902
                      0.014096
                      0.007552
 DT - feature_importance_vals
                     0.000000
                     0.040596
                     0.015862
                     0.000000
                     0.003847
                     0.002370
                     0.085292
                     0.018060
                     0.004584
                     0.016719
                   

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


 NN - feature_importance_vals
                     0.005629
                     0.007112
                     0.000870
                     0.031184
                     0.095627
                     0.013709
                     0.014097
                     0.010881
                     0.021225
                     0.001373
                     0.000854
                     0.030021
                     0.000839
                     0.001433
                     0.016697
                     0.000177
                     0.022542
                     0.001856
                     0.002006
                     0.000165
